In [1]:
%%capture
%run 01_Data_Preparation.ipynb

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model

# Hyperparameters

In [3]:
# hyperparameters
BATCH_SIZE = 1
EPOCHS = 10000
sequence_len = 4
n_features = 1 
# number of neurons in LSTM
num_units = 100

In [4]:
TRAIN_LENGTH = len(z_train)
#TEST_LENGTH = len(z_test)
VAL_LENGTH = len(z_val)

# RNN

In [5]:
def my_model(dropout_p=0.2):

    inputs = layers.Input((sequence_len, n_features), batch_size=BATCH_SIZE)
    x = inputs
    #x = layers.Rescaling(scale = 1./10) (x)
    x = layers.LSTM(100, return_sequences=True, stateful=True) (x)
    #x = layers.BatchNormalization() (x)
    x = layers.Dropout(dropout_p) (x)

    x = layers.LSTM(100, return_sequences=False, stateful=True) (x)
    #x = layers.BatchNormalization() (x)
    x = layers.Dropout(dropout_p) (x)

    x = layers.Dense(1) (x)
    #x = layers.Rescaling(scale = 100.) (x)
    return Model(inputs=inputs, outputs=x)

In [6]:
model = my_model()

2022-04-28 09:32:27.390418: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-28 09:32:27.444880: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-28 09:32:27.445079: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-28 09:32:27.445550: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(1, 4, 1)]               0         
                                                                 
 lstm (LSTM)                 (1, 4, 100)               40800     
                                                                 
 dropout (Dropout)           (1, 4, 100)               0         
                                                                 
 lstm_1 (LSTM)               (1, 100)                  80400     
                                                                 
 dropout_1 (Dropout)         (1, 100)                  0         
                                                                 
 dense (Dense)               (1, 1)                    101       
                                                                 
Total params: 121,301
Trainable params: 121,301
Non-trainable

# RNN + BNN

In [11]:
def model_uncertainity(dropout_p=0.2):

    inputs = layers.Input((sequence_len, n_features), batch_size=BATCH_SIZE)
    x = inputs
    #x = layers.Rescaling(scale = 1./10) (x)
    x = layers.LSTM(100, return_sequences=True, stateful=True) (x)
    #x = layers.BatchNormalization() (x)
    x = layers.Dropout(dropout_p) (x, training=True)

    x = layers.LSTM(100, return_sequences=True, stateful=True) (x)
    #x = layers.BatchNormalization() (x)
    x = layers.Dropout(dropout_p) (x, training=True)

    x = layers.Dense(1) (x)
    #x = layers.Rescaling(scale=100.)
    return Model(inputs=inputs, outputs=x)

In [12]:
model_u = model_uncertainity()

In [13]:
model_u.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(1, 4, 1)]               0         
                                                                 
 lstm_4 (LSTM)               (1, 4, 100)               40800     
                                                                 
 dropout_4 (Dropout)         (1, 4, 100)               0         
                                                                 
 lstm_5 (LSTM)               (1, 4, 100)               80400     
                                                                 
 dropout_5 (Dropout)         (1, 4, 100)               0         
                                                                 
 dense_2 (Dense)             (1, 4, 1)                 101       
                                                                 
Total params: 121,301
Trainable params: 121,301
Non-trainab